#### 1.Bibliothèques 

In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.metrics import zero_one_loss, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

#### 2. Chargement de données depuis le fichier 'fetal_health.csv'

In [5]:
data = pd.read_csv('fetal_health.csv')
data.shape

(2126, 22)

#### 3.  Correction de données manquantes : exemple de complétion de ces valeurs par la moyenne de la colonne
PS : Nous n'avons pas de valeurs manquantes donc ce bout de code ne va pas influencer notre code ou données  

In [6]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(data)
data=imp.transform(data) 

#### 4. Séprataion des attributs (Inputs) et de l'attribut objectif (Output)

In [7]:
NB_FEATURES=21
X=data[:,0:NB_FEATURES]         # Features: inputs (21 premières colonnes)
y=data[:,NB_FEATURES:].ravel()  # Target : output/ (Dernière colonne)

#### 5. Découpage de l'ensemble de données : 80% TRAINING, 20% TEST

In [8]:
TEST_SIZE = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,shuffle=True)

#### 6. Affihage des quelques informations concernant notre dataset 

In [9]:
print(f" >>> Taille du dataset / nombre d'instances :\t {data.shape[0]} ")
print(f" >>> Nombre d'attributs / Inputs : \t\t {NB_FEATURES} ")
print(f" >>> Taille du dataset d'entrainement : \t {X_train.shape[0]}")
print(f" >>> Taille du dataset de test : \t\t {X_test.shape[0]}")

 >>> Taille du dataset / nombre d'instances :	 2126 
 >>> Nombre d'attributs / Inputs : 		 21 
 >>> Taille du dataset d'entrainement : 	 1700
 >>> Taille du dataset de test : 		 426


#### 7. Normalisation des données

In [10]:
#Normalisation
scaler=preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

# Etape 2 : Paramètres par défaut
#### 8. Création d'un modèle de classification avec les paramètres par défaut de ScikitLearn + apprentissage du modèle

In [11]:
mlp =MLPClassifier(hidden_layer_sizes=(100,), activation='relu',  solver='adam',
                   alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001,
                   power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False,
                   warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                   validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, 
                   )

mlp.fit(X_train,y_train)
y_pred=mlp.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


#### 9. Métriques de classification

In [14]:
print(" Matrice de confusion = ") 
print(" "+str(confusion_matrix(y_pred,y_test)))
print(" Taux d'erreur = " +str(zero_one_loss(y_test, y_pred)))
print(" Exactitude (Accuracy)= " +str(accuracy_score(y_test,y_pred)))

 Matrice de confusion = 
 [[328  21   5]
 [  2  34   5]
 [  1   0  30]]
 Taux d'erreur = 0.07981220657277
 Exactitude (Accuracy)= 0.92018779342723


# Etape 3 : GridSearchCV

#### 10. Utilisation de la méthode GridSearchCV qui trouve facilement les valeurs optimales parmi les valeurs données pour les paramètres pertinents (activation, solver, learning_rate, max_iter)

In [15]:
from sklearn.model_selection import GridSearchCV

mlp_gs = MLPClassifier(max_iter=3000)
parameter_space = {
    'activation': ['relu', 'identity', 'logistic', 'tanh'],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'learning_rate': ['constant','adaptive', 'invscaling'],
    'max_iter': [200, 2000, 3000],
}

#n_jobs=-1 , -1 sert à utiliser tous les cœurs de processeur disponibles.
#cv = 10 est pour la validation croisée, ici cela signifie 10 fois la validation croisée stratifiée (K-fold)
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=10)

#Apprentissage / Entrainement de modèle
clf.fit(X, y)

#Affichage des scores pour toutes les combinaisons (modèles) possibles 
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

i=0
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    i=i+1    
print("Le nombre de combinaisons (modèles) possible est : %r " % (i)) 
print(" \n ")
print("|  Mean  |   Std   |  Combinaison / Modèle ")
print("-------------------------------------------------------------------------------------------------------------------")

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f    | %0.03f   | %r" % (mean, std, params))

Le nombre de combinaisons (modèles) possible est : 108 
 
 
|  Mean  |   Std   |  Combinaison / Modèle 
-------------------------------------------------------------------------------------------------------------------
0.811    | 0.090   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'adam'}
0.828    | 0.084   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'lbfgs'}
0.838    | 0.085   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'sgd'}
0.834    | 0.075   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 2000, 'solver': 'adam'}
0.787    | 0.140   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 2000, 'solver': 'lbfgs'}
0.837    | 0.088   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 2000, 'solver': 'sgd'}
0.834    | 0.083   | {'activation': 'relu', 'learning_rate': 'constant', 'max_iter': 3000, 'solver': 'adam'}
0.796    | 0.108   |

#### 11. Affichage du meilleur modèle (hyperparamètres optimaux parmi les valeurs données). 

In [16]:
print('Meilleurs paramètres trouvés : \n', clf.best_params_)

Meilleurs paramètres trouvés : 
 {'activation': 'logistic', 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'adam'}


#### 12. Test du meilleur modèle sur l'ensemble de test.

In [17]:
y_test, y_pred = y_test , clf.predict(X_test)

#### 13. Métriques de classification

In [20]:
print(" Matrice de confusion = ") 
print(" "+str(confusion_matrix(y_pred,y_test)))
print(" Taux d'erreur = " +str(zero_one_loss(y_test, y_pred)))
print(" Exactitude (Accuracy)= " +str(accuracy_score(y_test,y_pred)))

 Matrice de confusion = 
 [[164   2   2]
 [  0   0   0]
 [167  53  38]]
 Taux d'erreur = 0.5258215962441315
 Exactitude (Accuracy)= 0.47417840375586856
